Import Required Packages

In [28]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time

In [29]:
# set path 

In [30]:
path = "C:/Users/liqia/Spring2021-Project3-group-3-master/data//train_set"
image_dir = path+"images/"
pt_dir = path+"points/"
label_path = path+"label.csv"

read files

In [31]:
def read_data(path):
  
  # read labels
  labels = pd.read_csv(path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)

In [32]:
read_time_start=time.time()
X, Y = read_data(path)
print("Read the original dataset takes %s seconds" % round((time.time() - read_time_start),3))

Read the original dataset takes 52.557 seconds


In [33]:
X.shape, Y.shape 

((3000, 3003), (3000,))

Data Preprocessing Imbalanced Dataset (SMOTE)-oversamplig

In [34]:
print('majority class: %d' % np.sum(Y == 0))
print('minority class: %d' % np.sum(Y == 1))
#imbalanced dataset

majority class: 2402
minority class: 598


In [35]:
def data_preprocessing(X, Y, path):

  distances = X
  y = Y

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [36]:
Balanced_X, Blanced_Y = data_preprocessing(X, Y, path)

In [37]:
Balanced_X.shape, Blanced_Y.shape

((4804, 3003), (4804,))

split data

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd
import random
random.seed(2021)

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(Balanced_X, Blanced_Y,test_size=0.2,random_state=0)


In [40]:
#split train and test with size=0.20.2

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size=0.2,random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2400, 3003), (2400,), (600, 3003), (600,))

GBM model goes here (both original and updated)

In [ ]:
model_xgb = XGBClassifier()
model_xgb.fit(X_train, y_train)

In [ ]:
# make a prediction
X_test_flatten = test_images.reshape(600, 128*128*3)
y_pred = model.predict(X_test_flatten_smote)

In [ ]:
# print the confusion matrix
conf_mat = confusion_matrix(y_true=test_labels_smote, y_pred=y_pred)
print(conf_mat)

In [ ]:
print(classification_report(y_true=test_labels_smote, y_pred=y_pred))

In [ ]:
# Save best gbm model
pickle.dump(gbm_best, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/baseline_gbm.p",'wb'))


In [ ]:
#Save trained weighted SVM model
pickle.dump(weighted_svm_best, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/best_weighted_svm.p",'wb'))


SVM

In [59]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV,RepeatedStratifiedKFold,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import scipy.io
import pickle
import os, sys
from scipy.spatial.distance import pdist
import time 
import xlsxwriter
from sklearn.metrics import accuracy_score, classification_report,make_scorer, confusion_matrix,roc_auc_score
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
#from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn import ensemble
from sklearn.svm import SVC
import random
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input, Dropout
from keras.layers import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.utils import to_categorical

In [60]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 
grid_svm = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
grid_svm.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  28.5s
[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  24.6s
[CV 3/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  26.3s
[CV 4/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  25.7s
[CV 5/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  27.2s
[CV 1/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  26.8s
[CV 2/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  27.4s
[CV 3/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  26.9s
[CV 4/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  26.2s
[CV 5/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=  26.3s
[CV 1/5] END ..................C=0.1, gamma=0.01, kernel=rbf; total time=  27.4s
[CV 2/5] END ..................C=0.1, gamma=0.0

[CV 2/5] END ....................C=1000, gamma=1, kernel=rbf; total time=  24.3s
[CV 3/5] END ....................C=1000, gamma=1, kernel=rbf; total time=  24.1s
[CV 4/5] END ....................C=1000, gamma=1, kernel=rbf; total time=  24.5s
[CV 5/5] END ....................C=1000, gamma=1, kernel=rbf; total time=  24.3s
[CV 1/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=  24.2s
[CV 2/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=  24.2s
[CV 3/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=  24.3s
[CV 4/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=  24.3s
[CV 5/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=  24.2s
[CV 1/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=  24.0s
[CV 2/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=  24.0s
[CV 3/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=  23.3s
[CV 4/5] END ...............

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [61]:
print(grid_svm.best_params_)
print(grid_svm.best_estimator_)

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=10, gamma=0.0001)


In [62]:
#Train SVM using best parameters
svm_best = SVC(C=10,gamma=0.0001,kernel='rbf',probability=True) 
start_time=time.time()
svm_best.fit(X_train, y_train)

print("Training  model takes %s seconds" % round((time.time() - start_time),3))
print('Accuracy of SVM on test set: {:.3f}'.format(svm_best.score(X_test,y_test)))

start = time.time()
svm_pred = svm_best.predict(X_test)
end = time.time()

svm_predprob = svm_best.predict_proba(X_test)[:,1]

print("Predicting test data takes %s seconds" % round((end - start),3))
print('Classification error rate:', np.mean(np.array(y_test)!= svm_pred))
print('Classification report \n', classification_report(y_test, svm_pred))

print('Confusion Matrix \n', confusion_matrix(y_test, svm_pred))
print('AUC is: {:.4f}'.format(roc_auc_score(y_test, svm_predprob)))

Training  model takes 150.439 seconds
Accuracy of SVM on test set: 0.519
Predicting test data takes 12.722 seconds
Classification error rate: 0.48074921956295524
Classification report 
               precision    recall  f1-score   support

           0       0.92      0.05      0.09       484
           1       0.51      1.00      0.67       477

    accuracy                           0.52       961
   macro avg       0.72      0.52      0.38       961
weighted avg       0.72      0.52      0.38       961

Confusion Matrix 
 [[ 24 460]
 [  2 475]]
AUC is: 0.5340


In [76]:
#Save trained SVM model
pickle.dump(svm_best, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/best_svm.p",'wb'))

weighted SVM

In [66]:
weighted_svm = SVC(gamma = 'scale', class_weight = 'balanced')

#CV Weighted SVM 
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(weighted_svm, X, Y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % np.mean(scores))

Mean ROC AUC: 0.708


In [68]:
balance = [{0:598.0, 1:2402.0},{0:1,1:100}, {0:1,1:10}, {0:1,1:1}, {0:10,1:1}, {0:100,1:1}]
param_grid = dict(class_weight=balance)

grid_weightedsvm = GridSearchCV(estimator=weighted_svm, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
grid_weightedsvm.fit(X,Y)
grid_weightedsvm.best_params_
grid_weightedsvm.best_estimator_

SVC(class_weight={0: 598.0, 1: 2402.0})

In [69]:
grid_weightedsvm.best_params_
grid_weightedsvm.best_estimator_

SVC(class_weight={0: 598.0, 1: 2402.0})

In [70]:
print("Best: %f using %s" % (grid_weightedsvm.best_score_, grid_weightedsvm.best_params_))
# report all configurations
means = grid_weightedsvm.cv_results_['mean_test_score']
stds = grid_weightedsvm.cv_results_['std_test_score']
params = grid_weightedsvm.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.827703 using {'class_weight': {0: 598.0, 1: 2402.0}}
0.827703 (0.030973) with: {'class_weight': {0: 598.0, 1: 2402.0}}
0.792564 (0.035455) with: {'class_weight': {0: 1, 1: 100}}
0.782922 (0.031263) with: {'class_weight': {0: 1, 1: 10}}
0.798206 (0.034647) with: {'class_weight': {0: 1, 1: 1}}
0.796063 (0.030676) with: {'class_weight': {0: 10, 1: 1}}
0.796048 (0.030652) with: {'class_weight': {0: 100, 1: 1}}


In [73]:
weighted_svm_best = SVC(gamma = 'scale', class_weight ={0: 598.0, 1: 2402.0},probability=True)
start_time=time.time()
weighted_svm_best.fit(X, Y)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

print('Accuracy of weighted SVM on test set: {:.3f}'.format(weighted_svm_best.score(X,Y)))

start = time.time()
weighted_svm_pred = weighted_svm_best.predict(X_test)
end = time.time()
print(weighted_svm_pred[0:5,])

weighted_svm_predprob = weighted_svm_best.predict_proba(X_test)[:,1]

print("Predicting test data takes %s seconds" % round((end - start),3))
print('Classification error rate:', np.mean(np.array(y_test)!= weighted_svm_pred))
print('Classification report \n', classification_report(y_test, weighted_svm_pred))

print('Confusion Matrix \n', confusion_matrix(y_test, weighted_svm_pred))
print('AUC is: {:.4f}'.format(roc_auc_score(y_test, weighted_svm_predprob)))

Training  model takes 62.867 seconds
Accuracy of weighted SVM on test set: 0.863
[0 1 1 0 0]
Predicting test data takes 4.626 seconds
Classification error rate: 0.11446409989594172
Classification report 
               precision    recall  f1-score   support

           0       0.92      0.84      0.88       484
           1       0.85      0.93      0.89       477

    accuracy                           0.89       961
   macro avg       0.89      0.89      0.89       961
weighted avg       0.89      0.89      0.89       961

Confusion Matrix 
 [[408  76]
 [ 34 443]]
AUC is: 0.9465


In [80]:
#Save trained weighted SVM model
pickle.dump(weighted_svm_best, open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/best_weighted_svm.p",'wb'))

#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/best_weighted_svm.p",'rb'))

In [84]:
#Load SVM balanced model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/best_svm.p",'rb'))


SVC(C=10, gamma=0.0001, probability=True)